In [1]:
import h5py
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import glob
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import re

2024-05-12 11:28:41.409378: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 11:28:41.463584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
def plot_weights_biases_side_by_side(model1, model2):
    # Iterate over the parameters of the first model and collect names
    params1 = {name: param for name, param in model1.named_parameters()}
    params2 = {name: param for name, param in model2.named_parameters()}

    # Ensure both models have the same parameters for comparison
    assert params1.keys() == params2.keys(), "Models have different architectures"

    # Get the parameter names for plotting
    parameter_names = [name for name in params1.keys() if 'weight' in name]

    # Custom sorting function
    def custom_sort(name):
        # Extract the number after "conv"
        match = re.search(r'conv(\d+)', name)
        if match:
            return int(match.group(1))
        else:
            return float('inf')  # If no number found, place it at the end

    # Sort parameter_names by the number after "conv"
    parameter_names.sort(key=custom_sort)

    # Create traces for Model 1
    traces_model1 = []
    for name in parameter_names:
        trace = go.Violin(x=params1[name].data.cpu().numpy().flatten(), line_color='blue', name=f'{name} - Eigenvectors', side='negative')
        traces_model1.append(trace)

    # Create traces for Model 2
    traces_model2 = []
    for name in parameter_names:
        trace = go.Violin(x=params2[name].data.cpu().numpy().flatten(), line_color='green', name=f'{name} - Simulation data', side='positive')
        traces_model2.append(trace)

    # Create figure for Model 1
    fig_model1 = go.Figure(data=traces_model1 + traces_model2)
    fig_model1.update_traces(orientation='h', width=3, points=False)
    fig_model1.update_layout(title='Model Comparison', height=1000)  # Increase height to accommodate all data
    fig_model1.update_layout(xaxis_showgrid=False, xaxis_zeroline=False)

    fig_model1.show()

# Model: Eigenvectors

In [8]:
model_eigenvectors = torch.jit.load("model_eigenvectors.pt")
model_eigenvectors.eval()
model_simulation_data = torch.jit.load("model_kaneda.pt")
model_simulation_data.eval()
plot_weights_biases_side_by_side(model_eigenvectors, model_simulation_data)